# Forex Screening Strategies

This notebook demonstrates forex (currency pair) screening using tvscreener.

**Topics covered:**
- Major and minor pairs
- Technical analysis
- Volatility analysis
- Multi-timeframe analysis

In [ ]:
from tvscreener import ForexScreener
from tvscreener.field import ForexField
import pandas as pd

---
## Basic Forex Screening

In [ ]:
fs = ForexScreener()
df = fs.get()
print(f"Retrieved {len(df)} forex pairs with {len(df.columns)} columns")
df.head(10)

---
## Top Movers

In [ ]:
fs = ForexScreener()

fs.sort_by(ForexField.CHANGE_PERCENT, ascending=False)
fs.set_range(0, 30)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.CHANGE_PERCENT,
    ForexField.HIGH,
    ForexField.LOW
)

df = fs.get()
print(f"Top {len(df)} forex movers today")
df.head(15)

---
## USD Pairs

In [ ]:
fs = ForexScreener()
fs.search("USD")  # Pairs containing USD
fs.set_range(0, 100)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.CHANGE_PERCENT
)

df = fs.get()
print(f"Found {len(df)} USD pairs")
df.head(15)

---
## Technical Analysis

### RSI Oversold

In [ ]:
fs = ForexScreener()

fs.where(ForexField.RELATIVE_STRENGTH_INDEX_14 < 30)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.RELATIVE_STRENGTH_INDEX_14,
    ForexField.CHANGE_PERCENT
)
fs.sort_by(ForexField.RELATIVE_STRENGTH_INDEX_14, ascending=True)

df = fs.get()
print(f"Found {len(df)} oversold forex pairs (RSI < 30)")
df.head(15)

### RSI Overbought

In [ ]:
fs = ForexScreener()

fs.where(ForexField.RELATIVE_STRENGTH_INDEX_14 > 70)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.RELATIVE_STRENGTH_INDEX_14,
    ForexField.CHANGE_PERCENT
)
fs.sort_by(ForexField.RELATIVE_STRENGTH_INDEX_14, ascending=False)

df = fs.get()
print(f"Found {len(df)} overbought forex pairs (RSI > 70)")
df.head(15)

### Golden Cross Setup

In [ ]:
# Note: Field-to-field comparisons are NOT supported by TradingView API.
# Instead, retrieve data and filter with pandas:

fs = ForexScreener()

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.SIMPLE_MOVING_AVERAGE_50,
    ForexField.SIMPLE_MOVING_AVERAGE_200,
    ForexField.CHANGE_PERCENT
)

df = fs.get()

# Filter for golden cross setup using pandas
golden_cross = df[
    (df['Price'] > df['SMA 50']) &
    (df['SMA 50'] > df['SMA 200'])
]
print(f"Found {len(golden_cross)} pairs with golden cross setup")
golden_cross.head(15)

### MACD Analysis

In [ ]:
fs = ForexScreener()

# MACD bullish
fs.where(ForexField.MACD_LEVEL_12_26 > 0)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.MACD_LEVEL_12_26,
    ForexField.MACD_SIGNAL_12_26,
    ForexField.CHANGE_PERCENT
)

df = fs.get()
print(f"Found {len(df)} pairs with bullish MACD")
df.head(15)

### Stochastic Oversold

In [ ]:
fs = ForexScreener()

fs.where(ForexField.STOCHASTIC_K_14_3_3 < 20)
fs.where(ForexField.STOCHASTIC_D_14_3_3 < 20)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.STOCHASTIC_K_14_3_3,
    ForexField.STOCHASTIC_D_14_3_3,
    ForexField.CHANGE_PERCENT
)

df = fs.get()
print(f"Found {len(df)} pairs with oversold stochastic")
df.head(15)

---
## Volatility Analysis

### High ATR Pairs

In [ ]:
fs = ForexScreener()

fs.where(ForexField.AVERAGE_TRUE_RANGE_14 > 0.005)  # High ATR

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.AVERAGE_TRUE_RANGE_14,
    ForexField.VOLATILITY,
    ForexField.CHANGE_PERCENT
)
fs.sort_by(ForexField.AVERAGE_TRUE_RANGE_14, ascending=False)

df = fs.get()
print(f"Found {len(df)} high volatility pairs")
df.head(15)

---
## Multi-Timeframe Analysis

In [ ]:
fs = ForexScreener()

# Daily RSI moderate
fs.where(ForexField.RELATIVE_STRENGTH_INDEX_14.between(40, 60))

# 4-hour RSI oversold
rsi_4h = ForexField.RELATIVE_STRENGTH_INDEX_14.with_interval('240')
fs.where(rsi_4h < 35)

# 1-hour MACD bullish
macd_1h = ForexField.MACD_LEVEL_12_26.with_interval('60')
fs.where(macd_1h > 0)

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.RELATIVE_STRENGTH_INDEX_14,
    rsi_4h,
    macd_1h
)

df = fs.get()
print(f"Found {len(df)} multi-timeframe setups")
df.head(15)

---
## Specific Currency Pairs

In [ ]:
fs = ForexScreener()

# Major pairs
fs.symbols = {
    "query": {"types": []},
    "tickers": [
        "FX:EURUSD",
        "FX:GBPUSD",
        "FX:USDJPY",
        "FX:USDCHF",
        "FX:AUDUSD",
        "FX:USDCAD",
        "FX:NZDUSD"
    ]
}

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.CHANGE_PERCENT,
    ForexField.RELATIVE_STRENGTH_INDEX_14,
    ForexField.AVERAGE_TRUE_RANGE_14
)

df = fs.get()
df

---
## Performance Analysis

In [ ]:
fs = ForexScreener()

fs.where(ForexField.WEEKLY_PERFORMANCE > 1)  # Up 1%+ weekly

fs.select(
    ForexField.NAME,
    ForexField.PRICE,
    ForexField.CHANGE_PERCENT,
    ForexField.WEEKLY_PERFORMANCE,
    ForexField.MONTHLY_PERFORMANCE
)
fs.sort_by(ForexField.WEEKLY_PERFORMANCE, ascending=False)

df = fs.get()
print(f"Found {len(df)} pairs with strong weekly performance")
df.head(15)

---
## Select All Fields

In [ ]:
fs = ForexScreener()
fs.select_all()
fs.set_range(0, 10)

df = fs.get()
print(f"Retrieved {len(df.columns)} columns for {len(df)} pairs")
df.head()